In [17]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
import torch
from torch.utils.data import Dataset, DataLoader

data = [
    ("С днем рождения! Желаю счастья, здоровья и всего наилучшего!", "День рождения"),
    ("С Новым годом! Пусть этот год принесет вам радость и успех!", "Новый год"),
    ("С 8 Марта! Желаю весны, любви и красоты в вашей жизни!", "Международный женский день"),
    ("Счастливого Рождества! Пусть в вашем доме всегда будет тепло и уют!", "Рождество"),
    ("С Днем защитника Отечества! Желаю мира, силы духа и верных друзей!", "День защитника Отечества"),
    # Добавление нескольких примеров для каждого праздника для лучшего обучения
    ("Поздравляю с Днем рождения! Пусть каждый новый день приносит радость и удовлетворение!", "День рождения"),
    ("С наступающим Новым годом! Желаю, чтобы он был лучше предыдущего!", "Новый год"),
    ("С 8 Марта! Желаю счастья, здоровья и крепкой любви!", "Международный женский день"),
    ("Веселого Рождества! Пусть будет много смеха, радости и счастливых моментов!", "Рождество"),
    ("Поздравляю с Днем защитника Отечества! Желаю стойкости, отваги и уверенности в себе!", "День защитника Отечества"),
     # Добавление новых примеров
    ("С днем рождения! Пусть этот день станет настоящим днем счастья!", "День рождения"),
    ("С Новым годом! Желаю, чтобы этот год был плодотворным и наполнен успехом!", "Новый год"),
    ("С 8 Марта! Пусть этот день принесет радость и веселье!", "Международный женский день"),
    ("Счастливого Рождества! Пусть этот праздник будет для вас особенным!", "Рождество"),
    ("С Днем защитника Отечества! Желаю вам сил и вдохновения!", "День защитника Отечества"),
    # Дополнительные примеры
    ("С днем рождения! Желаю вам здоровья, радости и веселья!", "День рождения"),
    ("С Новым годом! Пусть этот год будет наполнен успехом и счастьем!", "Новый год"),
    ("С 8 Марта! Желаю вам всего самого лучшего на этот праздник!", "Международный женский день"),
    ("Счастливого Рождества! Пусть этот праздник принесет море подарков!", "Рождество"),
    ("С Днем защитника Отечества! Чтобы Вы всегда с мужеством шли по дороге своей жизни, с честью и достоинством встречали все проблемы и решали их легко и просто. !", "День защитника Отечества"),
]*12


# Подготовка данных
class GreetingDataset(Dataset):
    def __init__(self, tokenizer, data, max_length=100):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.attn_masks = []

        for item in data:
            encodings_dict = tokenizer('<|startoftext|>' + item[1] + ' [SEP] ' + item[0] + '<|endoftext|>', 
                                       truncation=True, max_length=max_length, padding="max_length")
            
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
max_length = 100
dataset = GreetingDataset(tokenizer, data, max_length=max_length)

loader = DataLoader(dataset, batch_size=2, shuffle=True)

# Настройка модели
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Обучение модели
optimizer = AdamW(model.parameters(), lr=5e-5)
model.train()

epochs = 15
for epoch in range(epochs):
    for idx, batch in enumerate(loader):
        optimizer.zero_grad()
        input_ids, attention_masks = batch
        input_ids = input_ids.to(device)
        attention_masks = attention_masks.to(device)

        outputs = model(input_ids, attention_mask=attention_masks, labels=input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        if idx % 10 == 0:
            print(f"Epoch: {epoch}, Loss: {loss.item()}")

# Сохранение модели
model.save_pretrained("./gpt2_greetings")
tokenizer.save_pretrained("./gpt2_greetings")

# Генерация поздравления



C:\Users\User\PycharmProjects\machinelearninng\venv\lib\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 0, Loss: 61.32307815551758
Epoch: 0, Loss: 3.4262874126434326
Epoch: 0, Loss: 3.0135669708251953
Epoch: 0, Loss: 2.762882947921753
Epoch: 0, Loss: 2.3703315258026123
Epoch: 0, Loss: 1.9588171243667603
Epoch: 0, Loss: 1.7172168493270874
Epoch: 0, Loss: 1.8814373016357422
Epoch: 0, Loss: 1.6060707569122314
Epoch: 0, Loss: 1.696831464767456
Epoch: 0, Loss: 1.5480051040649414
Epoch: 0, Loss: 1.3724414110183716
Epoch: 1, Loss: 1.4443495273590088
Epoch: 1, Loss: 1.539047360420227
Epoch: 1, Loss: 1.1785218715667725
Epoch: 1, Loss: 1.2436654567718506
Epoch: 1, Loss: 1.0143544673919678
Epoch: 1, Loss: 1.061852216720581
Epoch: 1, Loss: 1.093065619468689
Epoch: 1, Loss: 0.7129022479057312
Epoch: 1, Loss: 0.8494742512702942
Epoch: 1, Loss: 0.733397901058197
Epoch: 1, Loss: 0.8608652949333191
Epoch: 1, Loss: 0.8365013599395752
Epoch: 2, Loss: 0.5384209752082825
Epoch: 2, Loss: 0.8266909718513489
Epoch: 2, Loss: 0.7121158838272095
Epoch: 2, Loss: 0.6443985104560852
Epoch: 2, Loss: 0.493605762

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:

0: <|startoftext|>День рождения [SEP] С днем рождения! Желаю счастья, здоровья и всего наилучшего!



In [21]:
model.eval()
prompt = "<|startoftext|>Рождество [SEP]"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

sample_outputs = model.generate(
    generated, 
    do_sample=True,   
    top_k=50, 
    max_length=100,
    top_p=0.95, 
    num_return_sequences=1
)

print("Generated Text:\n")
for i, sample_output in enumerate(sample_outputs):
    text = tokenizer.decode(sample_output, skip_special_tokens=True)
    print("{}: {}\n".format(i, text))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text:

0: <|startoftext|>Рождество [SEP] Веселого Рождества! Пусть этот праздник будет для вас особенным!



In [11]:
prompt = "<|startoftext|>День рождения [SEP]"